# Surface Heat Flux/Ocean Heat Uptake

By: Ty Janoski
Updated: 12.30.21

## Set-up

In [1]:
# Import libraries and make svg images the default for inline plots

%matplotlib inline
%config InlineBackend.figure_format = "pdf"
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
from dask.diagnostics import ProgressBar

In [2]:
# create function for reading in cesm-LE output
# note: each ensemble member starts on a different year
# please keep this in mind when combining datasets
def read_in(exp,mon,ens,var):
    """
    Use xarray to read in a netCDF file.

    Keyword arguments:
    exp -- CO2 scenario
    mon -- starting month in which CO2 is altered
    ens -- ensemble number
    var -- model output variable
    """
    filein = '/dx01/janoski/cesm/output/b40.1850.cam5-lens.'+exp+'.'+str(
        f"{mon:02d}")+'.'+str(f"{ens:02d}")+'.h1_'+var+'.nc'
    return(xr.open_dataset(filein,chunks=None))

## Calculations

We define the net surface heat flux as the following:   
F$_{sfc}$ = F$_{LW}$ + F$_{SH}$ + F$_{LH}$ - F$_{SW}$   
where longwave, latent heat, and sensible heat flux are defined positive upwards, and shortwave is positive downwards.

In [4]:
m=1
# m=7

# iterate through every ensemble member
for e in range(1,101,1):
    if(e%10==0):
        print(e)
    # get control surface heat flux
    FLNS_ctrl = read_in('ctrl',m,e,'FLNS').FLNS
    FSNS_ctrl = read_in('ctrl',m,e,'FSNS').FSNS
    SHFLX_ctrl = read_in('ctrl',m,e,'SHFLX').SHFLX
    LHFLX_ctrl = read_in('ctrl',m,e,'LHFLX').LHFLX
    
    Fsfc_ctrl = FLNS_ctrl + SHFLX_ctrl + LHFLX_ctrl - FSNS_ctrl

    # same for 4xCO2
    FLNS_exp = read_in('4xCO2',m,e,'FLNS').FLNS
    FSNS_exp = read_in('4xCO2',m,e,'FSNS').FSNS
    SHFLX_exp = read_in('4xCO2',m,e,'SHFLX').SHFLX
    LHFLX_exp = read_in('4xCO2',m,e,'LHFLX').LHFLX

    Fsfc_exp = FLNS_exp + SHFLX_exp + LHFLX_exp - FSNS_exp

    # find difference
    dFLNS = FLNS_exp - FLNS_ctrl
    dFSNS = FSNS_ctrl - FSNS_exp # reverse to be consistent with up = positive
    dSHFLX = SHFLX_exp - SHFLX_ctrl
    dLHFLX = LHFLX_exp - LHFLX_ctrl
    dFsfc = Fsfc_exp - Fsfc_ctrl
    
    out = xr.merge([dFsfc.rename('dFsfc'),dFLNS.rename('dFLNS'),dFSNS.rename('dFSNS'),dSHFLX.rename('dSHFLX'),dLHFLX.rename('dLHFLX')])

    # save out
    pathout = '/dx02/janoski/cesm/vert_int_feedbacks/b40.1850.cam5-lens.'+str(
            f"{m:02d}")+'.'+str(f"{e:02d}")+'.h1_Fsfc.nc'
    
    out.to_netcdf(pathout)

10
20
30
40
50
60
70
80
90
100
